In [1]:
import pandas as pd

In [4]:
boros = ['bk', 'bx', 'mn', 'qn', 'si']

In [14]:
for b in boros:
    exec('%s_pop   = pd.read_csv("sources/%s/ACS_17_5YR_B01003_with_ann.csv", header=[0,1])' %(b,b))
    exec('%s_trans = pd.read_csv("sources/%s/ACS_17_5YR_B08301_with_ann.csv", header=[0,1])' %(b,b))

In [20]:
pop   = pd.concat([bk_pop, bx_pop, mn_pop, qn_pop, si_pop], ignore_index=True)
trans = pd.concat([bk_trans, bx_trans, mn_trans, qn_trans, si_trans], ignore_index=True)

In [32]:
pop.columns

MultiIndex(levels=[['GEO.display-label', 'GEO.id', 'GEO.id2', 'HD01_VD01', 'HD02_VD01'], ['Estimate; Total', 'Geography', 'Id', 'Id2', 'Margin of Error; Total']],
           labels=[[1, 2, 0, 3, 4], [2, 3, 1, 0, 4]])

In [80]:
pop['variable'] = "population"

In [81]:
pop.iloc[:,[1,3,4,5]].values

array([[36047000100, 4116, 366, 'population'],
       [36047000200, 1506, 264, 'population'],
       [36047000301, 4340, 480, 'population'],
       ..., 
       [36085031902, 5140, 558, 'population'],
       [36085032300, 1136, 130, 'population'],
       [36085990100, 0, 11, 'population']], dtype=object)

In [84]:
nyc_acs = pd.DataFrame(
    data=pop.iloc[:,[1,3,4,5]].values,
    columns=['geoid', 'value', 'moe', 'variable']
)

## Transportation

In [99]:
variables = [
    'Total:',
    'Car, truck, or van:',
    'Car, truck, or van: - Drove alone',
    'Car, truck, or van: - Carpooled',
    'Car, truck, or van: - Carpooled: - In 2-person carpool',
    'Car, truck, or van: - Carpooled: - In 3-person carpool',
    'Car, truck, or van: - Carpooled: - In 4-person carpool',
    'Car, truck, or van: - Carpooled: - In 5- or 6-person carpool',
    'Car, truck, or van: - Carpooled: - In 7-or-more-person carpool',
    'Public transportation (excluding taxicab):',
    'Public transportation (excluding taxicab): - Bus or trolley bus',
    'Public transportation (excluding taxicab): - Streetcar or trolley car (carro publico in Puerto Rico)',
    'Public transportation (excluding taxicab): - Subway or elevated',
    'Public transportation (excluding taxicab): - Railroad',
    'Public transportation (excluding taxicab): - Ferryboat',
    'Taxicab',
    'Motorcycle',
    'Bicycle',
    'Walked',
    'Other means',
    'Worked at home'
]

In [132]:
trans_acs = pd.DataFrame(columns=['geoid', 'value', 'moe', 'variable'])

In [133]:
for v in variables:
    exec("temp_df = trans.loc[:, ['Id2', 'Estimate; %s', 'Margin of Error; %s']]" %(v,v))
    temp_df['variable'] = v
    
    trans_acs = pd.concat([trans_acs, pd.DataFrame(
        data=temp_df.values,
        columns=['geoid', 'value', 'moe', 'variable']
    )], ignore_index=True)

In [146]:
trans_acs.replace(to_replace={
    'Total:': 'trans_total',
    'Car, truck, or van:': 'trans_auto_total',
    'Car, truck, or van: - Drove alone': 'trans_auto_solo',
    'Car, truck, or van: - Carpooled': 'trans_auto_carpool_total',
    'Car, truck, or van: - Carpooled: - In 2-person carpool': 'trans_auto_2',
    'Car, truck, or van: - Carpooled: - In 3-person carpool': 'trans_auto_3',
    'Car, truck, or van: - Carpooled: - In 4-person carpool': 'trans_auto_4',
    'Car, truck, or van: - Carpooled: - In 5- or 6-person carpool': 'trans_auto_5_or_6',
    'Car, truck, or van: - Carpooled: - In 7-or-more-person carpool': 'trans_auto_7_or_more',
    'Public transportation (excluding taxicab):': 'trans_public_total',
    'Public transportation (excluding taxicab): - Bus or trolley bus': 'trans_public_bus',
    'Public transportation (excluding taxicab): - Streetcar or trolley car (carro publico in Puerto Rico)': 'trans_public_streetcar',
    'Public transportation (excluding taxicab): - Subway or elevated': 'trans_public_subway',
    'Public transportation (excluding taxicab): - Railroad': 'trans_public_rail',
    'Public transportation (excluding taxicab): - Ferryboat': 'trans_public_ferry',
    'Taxicab': 'trans_taxi',
    'Motorcycle': 'trans_motorcycle',
    'Bicycle': 'trans_bicycle',
    'Walked': 'trans_walk',
    'Other means': 'trans_other',
    'Worked at home': 'trans_home'
}, inplace=True)

In [147]:
pd.concat([trans_acs, nyc_acs], ignore_index=True).to_csv('nyc_acs.csv')